# Network-based Gene Prioritization from GWAS Data
MED283: Network Biology & Biomedicine  
Nadia Arang & Kevin Chau

TODO:
1. Map rescores to old gene summaries table
2. SNP assignment according to regression analysis

In [1]:
%matplotlib inline

import ndex2
import networkx as nx
import pandas as pd
import numpy as np
from scipy.stats import spearmanr

## Initial data loading

GIANT
> Greene CS*, Krishnan A*, Wong AK*, Ricciotti E, Zelaya RA, Himmelstein DS, Zhang R, Hartmann BM, Zaslavsky E, Sealfon SC, Chasman DI, FitzGerald GA, Dolinski K, Grosser T, Troyanskaya OG. (2015). Understanding multicellular function and disease with human tissue-specific networks. Nature Genetics. 10.1038/ng.3259w.

PCNet
> Huang JK*, Carlin DE*, Yu MK, Zhang W, Kreisberg JF, Tamayo P, Ideker T. (2018). Systematic Evaluation of Molecular Networks for Discovery of Disease Genes. Cell Systems. 4 (6): 484-495.e5. doi: 10.1016/j.cels.2018.03.001

In [2]:
# Load network from server
# PCNet
# pc_nice = ndex2.create_nice_cx_from_server(server = "http://public.ndexbio.org", 
#                                            uuid = "f93f402c-86d4-11e7-a10d-0ac135e8bacf")

# GIANT; Brain-specific, 0.2-confidence filter
bn_nice = ndex2.create_nice_cx_from_server(server = "http://public.ndexbio.org", 
                                           uuid = "19677bff-6037-11e8-a4bf-0ac135e8bacf")

In [3]:
# Cast as networkx object
# First cast to pandas since networkx 2.1 is incompatible with ndex2
# nt_pd = pc_nice.to_pandas_dataframe()
nt_pd = bn_nice.to_pandas_dataframe()
net = nx.from_pandas_edgelist(nt_pd)

In [4]:
# Load gene summary table
gene_summaries_given = pd.read_csv("../src/gene_level_summary_stats_pmid_25056061.txt", 
                                   header = 0, sep = '\t', index_col = 1)
gene_summaries_given.head()

,Unnamed: 0,Chr,Gene Start,Gene End,nSNPs,TopSNP,TopSNP Pos,TopSNP P-Value,SNP Distance
Gene,,,,,,,,,
HIST1H4K,5905,6,27906930,27907284,8,rs34706883,27913234.0,5.071180e-10,6304.0
HIST1H2AK,5867,6,27913636,27914096,16,rs34706883,27913234.0,5.071180e-10,402.0
HIST1H2BN,5883,6,27914418,27914867,17,rs34706883,27913234.0,5.071180e-10,1184.0
HIST1H2AL,5868,6,27941085,27941555,10,rs13199772,27942064.0,7.053790e-10,979.0
HIST1H1B,5855,6,27942548,27943338,10,rs13199772,27942064.0,7.053790e-10,484.0


In [5]:
# Load snp summary table
snp_summaries = pd.read_csv("../src/snp_level_summary_stats_pmid_25056061.txt", 
                            header = 0, sep = '\t', index_col = 0, na_values = ".")
snp_summaries.head()

,hg18chr,bp,a1,a2,or,se,pval,info,ngt,CEUaf
snpid,,,,,,,,,,
rs3131972,1,742584,A,G,1.0257,0.0835,0.761033,0.1613,0,0.160550
rs3131969,1,744045,A,G,1.0221,0.0801,0.784919,0.2225,0,0.133028
rs3131967,1,744197,T,C,1.0227,0.0858,0.793520,0.2060,0,NaN
rs1048488,1,750775,T,C,0.9749,0.0835,0.761041,0.1613,0,0.836449
rs12562034,1,758311,A,G,1.0011,0.0756,0.987899,0.1856,3,0.092593


### Gene significance by normalized p-value

In [6]:
# Assign SNPs to genes based on +/- nkb window
nkb = 10

# Init new gene_summaries dataframe
gene_summaries = gene_summaries_given.loc[:, ["Chr", "Gene Start", "Gene End"]]
gene_summaries.head()

,Chr,Gene Start,Gene End
Gene,,,
HIST1H4K,6,27906930,27907284
HIST1H2AK,6,27913636,27914096
HIST1H2BN,6,27914418,27914867
HIST1H2AL,6,27941085,27941555
HIST1H1B,6,27942548,27943338


In [7]:
gene_scores = []
for gidx, gene in gene_summaries.iterrows():
    chrom = gene["Chr"]
    
    # filter for chromosome
    this_snps = snp_summaries[snp_summaries["hg18chr"] == chrom]
    
    # filter for upstream endpoint
    this_snps = this_snps[this_snps["bp"] >= gene["Gene Start"] - (nkb * 1000)]
    
    # filter for downstream endpoint
    this_snps = this_snps[this_snps["bp"] <= gene["Gene End"] + (nkb * 1000)]
    
    # Calculate adjusted bonferroni-adjusted p-value -> "gene score"
    pvals = [p * snp_summaries.shape[0] for p in this_snps.loc[:, "pval"].tolist()]
    adj_score = min(pvals) / len(pvals)
    gene_scores.append(adj_score)
gene_summaries["Gene Score"] = gene_scores

In [8]:
gene_summaries.head()

,Chr,Gene Start,Gene End,Gene Score
Gene,,,,
HIST1H4K,6,27906930,27907284,0.000079
HIST1H2AK,6,27913636,27914096,0.000040
HIST1H2BN,6,27914418,27914867,0.000037
HIST1H2AL,6,27941085,27941555,0.000088
HIST1H1B,6,27942548,27943338,0.000088


In [9]:
# Compare new gene list to given
glist_new = gene_summaries.sort_values(by = ["Gene Score"]).index.tolist()

In [10]:
glist_old = gene_summaries_given.sort_values(by = ["TopSNP P-Value"]).index.tolist()

In [11]:
spearmanr(glist_new, glist_old)

/usr/local/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


SpearmanrResult(correlation=0.002957765695483798, pvalue=0.7174935546409653)

In [12]:
gene_summaries[gene_summaries["Gene Score"] <= 0.05]

,Chr,Gene Start,Gene End,Gene Score
Gene,,,,
HIST1H4K,6,27906930,27907284,0.000079
HIST1H2AK,6,27913636,27914096,0.000040
HIST1H2BN,6,27914418,27914867,0.000037
HIST1H2AL,6,27941085,27941555,0.000088
HIST1H1B,6,27942548,27943338,0.000088
HIST1H3I,6,27947601,27948078,0.000088
HIST1H4L,6,27948904,27949268,0.000088
PGBD1,6,28357342,28378305,0.000052
HIST1H1E,6,26264537,26265322,0.000166


### Adding information to the network

In [13]:
nx.set_node_attributes(net, values = gene_summaries.T.to_dict())

### Overlapping the data

In [37]:
overlap_nodes = set(net.nodes) & set(gene_summaries.index)

In [39]:
len(overlap_nodes)

9896

In [40]:
net_distinct = [_ for _ in set(net.nodes) if _ not in overlap_nodes]
net.remove_nodes_from(net_distinct)

In [45]:
gene_summaries = gene_summaries[gene_summaries.index.isin(overlap_nodes)]

## Heat diffusion

With heat defined as p-value of top SNP per gene

### Diffusion kernel
Cowen et al. *Nature Review Genetics*, 2017

In [27]:
from scipy.linalg import *
import scipy.sparse
import math

In [46]:
alpha = 0.85
laplacian = nx.normalized_laplacian_matrix(net)

In [20]:
pd.DataFrame(laplacian.toarray()).head()

,0,1,2,3,4,5,6,7,8,9,...,14296,14297,14298,14299,14300,14301,14302,14303,14304,14305
0,1,-1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-1,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,-1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,-1,896,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
scipy.sparse.save_npz("../data/brain_giant_laplacian.npz", laplacian)

In [ ]:
dif_kernel = expm(-alpha * laplacian)

In [ ]:
new_pvec = np.dot(dif_kernel, gene_summaries["Gene Score"])

## Random walk with restarts

In [47]:
graph_kernel = alpha * inv(np.eye(laplacian.shape[0]) - ((1 - alpha) * laplacian))

In [49]:
gene_summaries["Gene ReScore"] = np.dot(graph_kernel, gene_summaries["Gene Score"])

In [51]:
gene_summaries

,Chr,Gene Start,Gene End,Gene Score,Gene ReScore
Gene,,,,,
HIST1H2BN,6,27914418,27914867,3.737463e-05,-1.007963e+00
HIST1H1B,6,27942548,27943338,8.837701e-05,1.142400e+01
PGBD1,6,28357342,28378305,5.162119e-05,-2.362811e+03
HIST1H1E,6,26264537,26265322,1.660777e-04,-1.786113e+03
HIST1H2BD,6,26266327,26279555,1.217903e-04,1.847831e+01
OR2B2,6,27987002,27988076,1.420720e-03,-2.030397e+03
EI24,11,124944507,124959785,2.604826e-03,2.214102e-03
ZNF184,6,27526505,27548873,1.869769e-03,2.409934e+00
MAD1L1,7,1903850,2239109,4.343130e-04,-5.789379e+01


In [55]:
gene_summaries.sort_values(by = ["Gene ReScore"], axis = 0, ascending = False).to_csv("../data/gene_list_output.csv")

## Validation